In [30]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

%matplotlib inline
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

hp = pd.read_csv("./data/HousePrice2.csv", index_col = 0)
hp_res= hp[hp['MSZoning'].isin(['FV','RH','RL','RM'])]
hp_res = hp_res.dropna()
hp_res.shape

Current working directory: /Users/chereowang27/NYCDSA/Machine_Learning_I/Machine_Learning_Project/Ames_Housing_ML


(2530, 83)

In [31]:
hp_res1 = hp_res.drop(['MapRefNo','PID', 'Prop_Addr'], \
                               axis = 1)
num_features = hp_res1._get_numeric_data()  \
 .drop(['MSSubClass','latitude','longitude','Unnamed: 0.1'], axis = 1)
num_features['bath'] = num_features['FullBath'] + 0.5*num_features['HalfBath']
numCol = num_features.columns

In [32]:
col = hp_res1.columns
catCol = [col for col in hp_res1.columns if col not in num_features.columns]
cat_features = hp_res[catCol].drop(['address', 'latitude', 'longitude','Unnamed: 0.1'],axis = 1)
cat_features.columns

Index(['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType',
       'SaleCondition'],
      dtype='object')

In [33]:
features = pd.concat([num_features, cat_features], axis = 1) \
           .drop(['FullBath', 'HalfBath', \
                  'BsmtFullBath', 'BsmtHalfBath', \
                  'BsmtFinSF1','SalePrice','ExterQual', \
                  'Condition1','Condition2', \
                  'BsmtFinType1','BsmtFinType2', \
                  'BsmtQual', 'GarageQual','MoSold',   \
                  'BsmtFinSF2', 'BsmtUnfSF'],axis = 1)

In [34]:
nb = features['Neighborhood'].unique()
def label_nb(row):
    if row['Neighborhood'] in (['NridgHt','NoRidge','Veenker','StoneBr']):      
        return 'N'
    if row['Neighborhood']      \
             in (['Blmngtn','Gilbert','OldTown', 'NWAmes','NAmes','Blmngtn','BrDale','Somerst','BrkSide','Greens','NPkVill']):
        return 'NE'
    if row['Neighborhood']      \
            in (['CollgCr','SWISU', 'Edwards','SawyerW','Sawyer','ClearCr']):
        return 'W'                      
    if row['Neighborhood'] in (['Crawfor','MeadowV','Timber','Mitchel','IDOTRR']):     
        return 'S'
    
features['Location']=features.apply(lambda row: label_nb(row), axis = 1)
features = features.drop(['Neighborhood'], axis = 1)
features1 = features    

In [35]:
catCol = [col for col in features.columns if col not in num_features.columns]
features= pd.get_dummies(features, columns = catCol, drop_first = True)

y = hp_res['SalePrice']
hp_res['log_Price'] = np.log10(hp_res['SalePrice'])
logy = hp_res['log_Price']

In [36]:
from sklearn import linear_model, datasets
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import time

ridge = Ridge()
lasso = Lasso()
# net = ElasticNet()
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
# X_train, X_test, y_train, y_test = train_test_split(features, y, test_size = 0.3, random_state = 0)

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(features, logy, test_size = 0.3, random_state = 0)

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

lasso_pipe = Pipeline([
    ('standardize', StandardScaler()),
    ('lasso',lasso)
])
params = {
#     'lasso__alpha': [0.0001, 0.001, 0.1, 1, 10]      .001
     'lasso__alpha': np.logspace(-4,-3,100)        # .001
#    'lasso_max_iter': [5000],
     }
kfold = KFold(n_splits = 3, shuffle = True)
gs_log = GridSearchCV(lasso_pipe, param_grid = params, scoring = 'r2', cv = kfold)

start = time.time()
gs_log.fit(X_train_log, y_train_log)
print("Estimator which gave the highest score: ", gs_log.best_estimator_)
print("Mean cross-validated score of the best_estimator: ", gs_log.best_score_)
print("The best parameters are: ", gs_log.best_params_)

Estimator which gave the highest score:  Pipeline(steps=[('standardize', StandardScaler()),
                ('lasso', Lasso(alpha=0.0006734150657750821))])
Mean cross-validated score of the best_estimator:  0.908625587051581
The best parameters are:  {'lasso__alpha': 0.0006734150657750821}


In [38]:
# Run Lasso Regression with the best alpha from Grid search .001

#for alpha in alphas:
lasso.set_params(alpha = .001)
lasso.fit(X_train_log, y_train_log)
lasso_coefs = dict(zip(features.columns, lasso.coef_))
lasso_coefs

lasso_coefs_keep = {k:v for (k,v) in lasso_coefs.items() if v !=0}
lasso_coefs_keep
 

/Users/chereowang27/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.168649442268886, tolerance: 0.004849049232745547
  model = cd_fast.enet_coordinate_descent(


{'GrLivArea': 0.00019163748117907289,
 'LotFrontage': 0.00028904103169879666,
 'LotArea': 1.1357983009707266e-06,
 'OverallQual': 0.0305790731645165,
 'OverallCond': 0.023206655847639535,
 'YearBuilt': 0.0012317235675402983,
 'YearRemodAdd': 0.0004262687109342517,
 'MasVnrArea': 1.1759296623682326e-05,
 'TotalBsmtSF': 6.351638997032007e-05,
 '1stFlrSF': -6.519813143370877e-05,
 '2ndFlrSF': -6.776185265256836e-05,
 'LowQualFinSF': -0.00012520360882758538,
 'BedroomAbvGr': -0.008936999929735467,
 'KitchenAbvGr': -0.007960129311996281,
 'Fireplaces': 0.014312593698608392,
 'GarageYrBlt': 1.110772435366692e-05,
 'GarageArea': 5.459894543550582e-05,
 'WoodDeckSF': 3.589607311420987e-05,
 'OpenPorchSF': 1.85295299616461e-05,
 'EnclosedPorch': 8.382934734386028e-05,
 '3SsnPorch': -7.430699461557589e-06,
 'ScreenPorch': 0.00012971487928604855,
 'PoolArea': -7.12703109744648e-05,
 'MiscVal': -6.686543760321962e-07,
 'YrSold': -0.0008702650272686956,
 'MSSubClass_30': -0.0012853507804278955,
 'M